## Importing Packages

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.transforms import InterpolationMode
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchsummary import summary
import torchvision
from torchvision.models import squeezenet1_1, SqueezeNet1_1_Weights
from tqdm import tqdm
import os
from torch.optim.lr_scheduler import StepLR
import random
import numpy as np

from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: NVIDIA L4


In [21]:
# Set a random seed for reproducibility
def set_seed(seed_value=42):
    random.seed(seed_value)       # Python random module
    np.random.seed(seed_value)    # Numpy module
    torch.manual_seed(seed_value) # Torch
    os.environ['PYTHONHASHSEED'] = str(seed_value)  # Environment variable

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  # if using multi-GPU
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(24)

## Importing the Dataset

In [37]:
'TODO: Define transformations - crop or resize'
transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for pre-trained models
])

"""

In order to load the datasets from the shared folder, go to google drive, right click the shared folder, and create a shortcut
to somewhere in your drive.

"""

# Ben's dataset paths
ben_train_dataset_path = "/content/gdrive/MyDrive/24S Classes/Deep Learning/COSC78 Final Project/Train Data"
ben_validation_dataset_path = "/content/gdrive/MyDrive/24S Classes/Deep Learning/COSC78 Final Project/Validation Data"
ben_test_dataset_path = "/content/gdrive/MyDrive/24S Classes/Deep Learning/COSC78 Final Project/Test Data"


# Dawson's dataset paths
daw_train_dataset_path = '/content/gdrive/MyDrive/COSC78 Final Project/Train Data'
daw_validation_dataset_path = '/content/gdrive/MyDrive/COSC78 Final Project/Validation Data'
daw_test_dataset_path = '/content/gdrive/MyDrive/COSC78 Final Project/Test Data'

# Will's dataset paths
will_train_dataset_path = '/content/gdrive/MyDrive/COSC78/COSC78 Final Project/Train Data'
will_validation_dataset_path = '/content/gdrive/MyDrive/COSC78/COSC78 Final Project/Validation Data'
will_test_dataset_path = '/content/gdrive/MyDrive/COSC78/COSC78 Final Project/Test Data'

# Brian's dataset paths
bri_train_dataset_path = '/content/gdrive/MyDrive/Algorithms - Collab/CS 78/COSC78 Final Project/Train Data'
bri_val_dataset_path = '/content/gdrive/MyDrive/Algorithms - Collab/CS 78/COSC78 Final Project/Validation Data'
bri_test_dataset_path = '/content/gdrive/MyDrive/Algorithms - Collab/CS 78/COSC78 Final Project/Test Data'

#% Dataset paths in use %# (currently brian's)
train_dataset_path = bri_train_dataset_path
validation_dataset_path = bri_val_dataset_path
test_dataset_path = bri_test_dataset_path


generator = torch.Generator(device=device)


with torch.device(device):
    # Setup datasets using ImageFolder
    train_dataset = datasets.ImageFolder(train_dataset_path, transform=transform)
    val_dataset = datasets.ImageFolder(validation_dataset_path, transform=transform)
    test_dataset = datasets.ImageFolder(test_dataset_path, transform=transform)

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True, generator=generator)
    val_loader = DataLoader(val_dataset, batch_size=2048, shuffle=False, generator=generator)
    test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False, generator=generator)

## Import and modify the model

In [33]:
# Load the pre-trained SqueezeNet 1_1 model
pretrained_weights = squeezenet1_1(pretrained=True)
squeezenet = pretrained_weights

# Freeze all the parameters for fine-tuning
for param in squeezenet.parameters():
    param.requires_grad = False

# Replace the classifier head
num_classes = 3  # Number of classes in your dataset
squeezenet.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
squeezenet.classifier[1].requires_grad_ = True

# Print the model summary
# summary(squeezenet, input_size=(3, 224, 224))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Fine tune the model

In [34]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(squeezenet.parameters())

In [14]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(squeezenet.parameters())
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # Decreases the learning rate by a factor of 0.1 every 10 epochs
torch.manual_seed(24)

In [35]:
def train_and_validate(model, train_loader, val_loader, optimizer, loss_func, epochs=25):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    model.to(device)

    history = {'train_loss': [], 'val_loss': []}

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs} - Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)

        train_loss = train_loss / len(train_loader.dataset)
        history['train_loss'].append(train_loss)

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{epochs} - Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = loss_func(outputs, labels)
                val_loss += loss.item() * images.size(0)

        val_loss = val_loss / len(val_loader.dataset)
        history['val_loss'].append(val_loss)

        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

    return history

In [ ]:
with torch.device(device):
  history = train_and_validate(squeezenet, train_loader, val_loader, optimizer, loss_func, epochs=25)

Using device: cuda


Epoch 1/25 - Training:  58%|█████▊    | 7/12 [27:15<24:26, 293.28s/it]

In [16]:
history = train_and_validate(squeezenet, train_loader, val_loader, optimizer, scheduler, loss_func, epochs=25)

In [ ]:
# Plotting training and validation accuracy
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.plot(history['train_accuracy'], label='Train Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

# Plotting training and validation F1 score
plt.subplot(1, 3, 2)
plt.plot(history['train_f1'], label='Train F1 Score')
plt.plot(history['val_f1'], label='Validation F1 Score')
plt.title('Model F1 Score')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend()

# Plotting training and validation loss
plt.subplot(1, 3, 3)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()